In [2]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import xarray as xr
import os
import shutil
import time
from glob import glob

def copy_binaries(dir_binaries,dir_run):
    print('Copy binaries from %s' % dir_binaries)

    shutil.copyfile(os.path.join(dir_binaries,'clm'), os.path.join(dir_run,'clm'))
    os.chmod(os.path.join(dir_run,'clm'),0o755) # read/execute permissions for all

    shutil.copyfile(os.path.join(dir_binaries,'parflow'), os.path.join(dir_run,'parflow'))
    os.chmod(os.path.join(dir_run,'parflow'),0o755) # read/execute permissions for all

def adjust_clm_files(dir_setup,dir_run,settings_clm):
    with open(os.path.join(dir_setup,'namelists/lnd.stdin'), 'r') as file :
      filedata = file.read()

    time_seconds_start = (settings_clm['datetime_start'].hour*60*60 + 
                          settings_clm['datetime_start'].minute*60 + 
                          settings_clm['datetime_start'].second)
    
    filedata = filedata.replace('__time_totalrun__', '%i'% (settings_clm['t_total'].total_seconds()/(60*60*24)) ) #in days
    filedata = filedata.replace('__time_clm_dump__', '%i'% (settings_clm['t_dump'].total_seconds()/(60*60)) ) #in hours
    filedata = filedata.replace('__time_couple__', '%i'% settings_clm['t_couple'].total_seconds() )  #in seconds  
    filedata = filedata.replace('__date_start__', '%s'% str(settings_clm['datetime_start'].date()).replace('-','') )
    filedata = filedata.replace('__time_start__', '%i'% time_seconds_start )    
    filedata = filedata.replace('__dir_forcing__', '%s'% settings_clm['dir_forcing'])
    filedata = filedata.replace('__clm_nx__', '%s'% settings_clm['nx'])
    filedata = filedata.replace('__clm_ny__', '%s'% settings_clm['ny'])
    filedata = filedata.replace('__file_restart__', '%s'% settings_clm['file_restart'])
    filedata = filedata.replace('__dir_clmin__', '%s'% os.path.join(dir_setup,'input_clm') )

    with open(os.path.join(dir_run,'lnd.stdin'), 'w') as file:
      file.write(filedata)
    os.chmod(os.path.join(dir_run,'lnd.stdin'),0o755) # read/execute permissions for all

def adjust_parflow_files(dir_setup,dir_run,dir_build,dir_bin,dir_real,settings_pfl):
    for file_ in ['input_pf/ascii2pfb_slopes.tcl','input_pf/ascii2pfb_SoilInd.tcl','input_pf/ascii2pfb_Ks.tcl','namelists/coup_oas.tcl']:

        with open( os.path.join(dir_setup,file_) , 'r') as file :
          filedata = file.read()

        filedata = filedata.replace('__nprocx_pfl_bldsva__', '%i'%settings_pfl['n_proc_pfl_x'])
        filedata = filedata.replace('__nprocy_pfl_bldsva__', '%i'%settings_pfl['n_proc_pfl_y'])
        filedata = filedata.replace('__pfl_dx__', settings_pfl['dx'])
        filedata = filedata.replace('__pfl_dy__', settings_pfl['dy'])
        filedata = filedata.replace('__pfl_nx__', '%i'%settings_pfl['nx'])
        filedata = filedata.replace('__pfl_ny__', '%i'%settings_pfl['ny'])

        filedata = filedata.replace('__dir_bin__', '%s'%dir_bin)
        
        filedata = filedata.replace('__dir_pfl__', '%s'%os.path.join(dir_build,'parflow') )
        filedata = filedata.replace('__dir_pfin__', '%s'% os.path.join(dir_setup,'input_pf') )
        filedata = filedata.replace('__dir_run__', '%s'% dir_run )
        filedata = filedata.replace('__dir_real__', '%s'%dir_real)
        
        filedata = filedata.replace('__time_totalrun__', '%i'% (settings_pfl['t_total'].total_seconds()/(60*60)) )
        filedata = filedata.replace('__time_pfl_dump__', '%.1f'% (settings_pfl['t_dump'].total_seconds()/(60*60)) )
        filedata = filedata.replace('__time_couple__', '%.2f'% (settings_pfl['t_couple'].total_seconds()/(60*60)) )   
        filedata = filedata.replace('__filename_pfl_out__', settings_pfl['filename_pfl_out'] )

        filedata = filedata.replace('__icpres_type__', '%s'%settings_pfl['icpres_type'])
        filedata = filedata.replace('__geom_icpres_valorfile__', '%s'%settings_pfl['geom_icpres_valorfile'])
        filedata = filedata.replace('__geom_icpres_val__', '%s'%settings_pfl['geom_icpres_val'])
        
        
        with open(os.path.join(dir_run,os.path.basename(file_)), 'w') as file:
          file.write(filedata)
        os.chmod(os.path.join(dir_run,os.path.basename(file_)),0o755)

def make_parflow_executable(dir_run,dir_build):
    str_cmd = '''
    source %s
    tclsh %s
    tclsh %s
    tclsh %s
    tclsh %s
    ''' % (os.path.join(dir_build,'bldsva/machines/JUWELS/loadenvs.Intel'),
           os.path.join(dir_run,'ascii2pfb_slopes.tcl'),
           os.path.join(dir_run,'ascii2pfb_SoilInd.tcl'),
           os.path.join(dir_run,'ascii2pfb_Ks.tcl'),
           os.path.join(dir_run,'coup_oas.tcl'))

    os.system(str_cmd)    

def adjust_oasis_files(dir_setup,dir_run,settings_clm,settings_pfl):
    with open(os.path.join(dir_setup,'namelists/namcouple_pfl_clm'), 'r') as file :
      filedata = file.read()

    filedata = filedata.replace('__runTime__', '%i'% settings_clm['t_total'].total_seconds())

    filedata = filedata.replace('__cplfreq__', '%i'% settings_clm['t_couple'].total_seconds())
    filedata = filedata.replace('__ngpflx__', '%i'% settings_clm['nx'])
    filedata = filedata.replace('__ngpfly__', '%i'% settings_clm['ny'])
    filedata = filedata.replace('__ngclmx__', '%i'% (settings_clm['nx']*settings_clm['ny'])) #the clm grid is 'flattened' in oasis
    filedata = filedata.replace('__ngclmy__', '%i'% 1)

    file_out = os.path.join(dir_run,'namcouple')
    # Write the file out again
    with open(file_out, 'w') as file:
      file.write(filedata)

    os.chmod(file_out,0o755) # read/execute permissions for all
    
def copy_oasis_files(dir_setup,dir_run):
    shutil.copy( os.path.join(dir_setup,'input_oas/rmp_gclm_to_gpfl_DISTWGT.nc'), dir_run )
    shutil.copy( os.path.join(dir_setup,'input_oas/rmp_gpfl_to_gclm_BILINEAR.nc'), dir_run )
    shutil.copy( os.path.join(dir_setup,'input_oas/clmgrid.nc'), dir_run )
    
def adjust_run_files(dir_setup,dir_run,settings_clm,settings_pfl,settings_sbatch):

    n_proc_pfl = settings_pfl['n_proc_pfl_x']*settings_pfl['n_proc_pfl_y']*settings_pfl['n_proc_pfl_z']
    n_proc_clm = settings_clm['n_proc_clm']
    proc_pfl_0 = 0
    proc_pfl_n = n_proc_pfl - 1 
    proc_clm_0 = n_proc_pfl
    proc_clm_n = n_proc_pfl + n_proc_clm - 1

    with open( os.path.join(dir_setup,'namelists/slm_multiprog_mapping.conf') , 'r') as file :
      filedata = file.read()

    filedata = filedata.replace('__pfl0__', '%i'%proc_pfl_0)
    filedata = filedata.replace('__pfln__', '%i'%proc_pfl_n)
    filedata = filedata.replace('__clm0__', '%i'%proc_clm_0)
    filedata = filedata.replace('__clmn__', '%i'%proc_clm_n)
    filedata = filedata.replace('__filename_pfl__', '%s'%settings_pfl['filename_pfl_out'])

    with open(os.path.join(dir_run,'slm_multiprog_mapping.conf'), 'w') as file:
      file.write(filedata)
    os.chmod(os.path.join(dir_run,'slm_multiprog_mapping.conf'),0o755)

    
    n_nodes = int(np.ceil( (n_proc_pfl + n_proc_clm) / 48 )) # 48 cores per node on JUWEL
    
    with open( os.path.join(dir_setup,'namelists/tsmp_slm_run.bsh') , 'r') as file :
      filedata = file.read()

    filedata = filedata.replace('__ntasks__', '%i'%(n_proc_pfl + n_proc_clm))
    filedata = filedata.replace('__nnodes__', '%i'%n_nodes)
    filedata = filedata.replace('__stime__', settings_sbatch['sbatch_time'])
    filedata = filedata.replace('__spart__', settings_sbatch['sbatch_partition'])
    filedata = filedata.replace('__sacc__', settings_sbatch['sbatch_account'])

    with open(os.path.join(dir_run,'tsmp_slm_run.bsh'), 'w') as file:
      file.write(filedata)
    os.chmod(os.path.join(dir_run,'tsmp_slm_run.bsh'),0o755)
    
def start_run(dir_build):
    # os.chdir(dir_run)
    # sbatch_file = os.path.join(dir_run,'tsmp_slm_run.bsh')
    sbatch_file = 'tsmp_slm_run.bsh'
    # os.system('sbatch %s' % sbatch_file)
    
    str_cmd = '''
    source %s
    sbatch %s
    ''' % (os.path.join(dir_build,'bldsva/machines/JUWELS/loadenvs.Intel'),
           sbatch_file)
    os.system(str_cmd)    
    
def wait_for_run(dir_run,settings_sbatch):
    while not os.path.exists(os.path.join(dir_run,'ready.txt')):
        print('Still running...')
        time.sleep(settings_sbatch['sbatch_check_sec'])
        
def move_and_link(to_link,folder_store,delete_existing=True):
    # move file to folder_store, and keep a symbolic link
    if type(to_link)==list:
        for file in to_link:
            if not os.path.islink(file):
                shutil.move( file, folder_store)

                # link files to rundir
                file_ln = os.path.join(folder_store,os.path.basename(file) )
                os.symlink(file_ln, file ) #verify this

    elif type(to_link)==str: # in this case 'to_link' is a single file or a folder
        if not os.path.islink(to_link):
            if delete_existing and os.path.exists(folder_store):
                shutil.rmtree(folder_store)
            
            shutil.move( to_link, folder_store)
            print('File/dir moved to storage: %s' % folder_store)

            # link files/folder back into the rundir
            file_ln = os.path.join(folder_store,os.path.basename(to_link) )
            print('Linking file/dir from %s to %s' % (file_ln,to_link))
            os.symlink(file_ln, to_link )   
             
    else:
        raise RuntimeError
    
def remove_misc_files(dir_run):
    [os.remove(file) for file in glob(os.path.join(dir_run,'CLMSAT*.nc'))]
    [os.remove(file) for file in glob(os.path.join(dir_run,'CLMFLX*.nc'))]
    [os.remove(file) for file in glob(os.path.join(dir_run,'PFLSAT*.nc'))]
    [os.remove(file) for file in glob(os.path.join(dir_run,'PFLSAT*.nc'))]
    [os.remove(file) for file in glob(os.path.join(dir_run,'debug*'))]
    [os.remove(file) for file in glob(os.path.join(dir_run,'nout.*'))]
    [os.remove(file) for file in glob(os.path.join(dir_run,'rmp*'))]

In [2]:
from settings import settings_run,settings_clm,settings_pfl,settings_sbatch, date_results_binned
settings_run['dir_real'] = '/p/project/cjibg36/kaandorp2/TSMP_patched/DA_tsmp_cordex_111x108/20190101-20191230/i000/R000'
date_results_iter = date_results_binned[0]
date_results_iter

[[Timestamp('2019-01-01 12:00:00', freq='3D'),
  Timestamp('2019-01-04 12:00:00', freq='3D'),
  Timestamp('2019-01-07 12:00:00', freq='3D'),
  Timestamp('2019-01-10 12:00:00', freq='3D'),
  Timestamp('2019-01-13 12:00:00', freq='3D'),
  Timestamp('2019-01-16 12:00:00', freq='3D'),
  Timestamp('2019-01-19 12:00:00', freq='3D'),
  Timestamp('2019-01-22 12:00:00', freq='3D'),
  Timestamp('2019-01-25 12:00:00', freq='3D'),
  Timestamp('2019-01-28 12:00:00', freq='3D'),
  Timestamp('2019-01-31 12:00:00', freq='3D')]]

In [3]:
## Outside of main loop    
# '''
#  1) Copy the folder template to the setup location if the destination does not exist
# '''
# if not os.path.exists(settings_run['dir_setup']):
#     print('Copying folder template from %s to %s' % (settings_run['dir_template'],settings_run['dir_setup']) )
#     shutil.copytree(settings_run['dir_template'],settings_run['dir_setup'])
# else:
#     print('Continuing simulation in %s' % settings_run['dir_setup'])

# # print('Copying folder template from %s to %s' % (settings_run['dir_template'],settings_run['dir_setup']) )
# # shutil.copytree(settings_run['dir_template'],settings_run['dir_setup'])    
# os.chdir(settings_run['dir_setup'])
        
      
        
'''
 2) Loop through the restart dates, if the folder does not exist create and setup run

'''
for i1,date_list in enumerate(date_results_iter):
    
    if i1==0 and not settings_run['init_restart']: #'Cold' run
        flag_restart = False
        settings_clm['file_restart'] = ''
        settings_pfl['icpres_type'] = 'HydroStaticPatch'
        settings_pfl['geom_icpres_valorfile'] = 'Value'
        settings_pfl['geom_icpres_val'] = '-2.0'
    elif i1 ==0 and settings_run['init_restart']: #provide a IC file from another run
        print('Restarting run from %s and %s' % (settings_clm['IC_file'],settings_pfl['IC_file']) )
        flag_restart = True
        settings_clm['file_restart'] = '%s' % settings_clm['IC_file'] 
        settings_pfl['icpres_type'] = 'NCFile'
        settings_pfl['geom_icpres_valorfile'] = 'FileName'
        settings_pfl['geom_icpres_val'] = '"%s"' % settings_pfl['IC_file']
    else: #base the initial conditions on restart file from previous date
        flag_restart = True
        date_list_prev = date_results_iter[i1-1]
        
        date_start_prev = date_list_prev[0]
        date_end_prev = date_list_prev[-1]
        str_date_prev = str(date_start_prev.date()).replace('-','') + '-' + str(date_end_prev.date()).replace('-','')
        if settings_run['spinup']:
            str_spinup_prev = '%3.3i_' % (i1-1)
        else:
            str_spinup_prev = ''
        dir_run_prev = os.path.join(settings_run['dir_real'],'run_%s%s' % (str_spinup_prev,str_date_prev) )
    
        files_clm_prev = sorted(glob(os.path.join(dir_run_prev,'clm.clm2.r.*.nc') ))
        assert len(files_clm_prev) == (len(date_list_prev)-1), 'Check if previous run crashed, available CLM restart files: %s' % files_clm_prev
        settings_pfl['icpres_type'] = 'NCFile'
        settings_pfl['geom_icpres_valorfile'] = 'FileName'    
        settings_clm['file_restart'] = files_clm_prev[-1]
        settings_pfl['geom_icpres_val'] = sorted(glob(os.path.join(dir_run_prev,
                                                                   'cordex%ix%i_%s.out.0*' % (settings_pfl['nx'],settings_pfl['ny'],str_date_prev))))[-1]
        
        
    date_start_sim = date_list[0]
    date_end_sim = date_list[-1]

    ## Define the run directory, as 'run_{start_date}-{end_date}', or 'run_{integer}_{start_date}-{end_date}' in the case of a spinup with the same dates repeated
    str_date = str(date_start_sim.date()).replace('-','') + '-' + str(date_end_sim.date()).replace('-','')
    if settings_run['spinup']:
        assert type(settings_run['spinup']) == int, 'Spinup needs to be False or an integer' 
        print('Running in spinup mode! Repeating %i times' % settings_run['spinup'])
        str_spinup = '%3.3i_' % i1
    else:
        str_spinup = ''
    dir_run = os.path.join(settings_run['dir_real'],'run_%s%s' % (str_spinup,str_date) )
    
    ## Main loop: if the given run directory does not exist prepare and submit the run
    if not os.path.exists(dir_run):
        print('Preparing simulation in %s' % dir_run )
        os.mkdir(dir_run)

        copy_binaries(settings_run['dir_binaries'],dir_run)
        
        settings_clm['t_total'] = date_end_sim-date_start_sim
        settings_clm['datetime_start'] = date_start_sim
        adjust_clm_files(settings_run['dir_setup'],dir_run,settings_clm)
        
        settings_pfl['t_total'] = date_end_sim-date_start_sim
        settings_pfl['filename_pfl_out'] = 'cordex%ix%i_%s' % (settings_pfl['nx'],settings_pfl['ny'],str_date)
        adjust_parflow_files(settings_run['dir_setup'],dir_run,
                             settings_run['dir_build'],settings_run['dir_binaries'],
                             settings_run['dir_real'],settings_pfl)
        make_parflow_executable(dir_run,settings_run['dir_build'])
        
        adjust_oasis_files(settings_run['dir_setup'],dir_run,settings_clm,settings_pfl)
        copy_oasis_files(settings_run['dir_setup'],dir_run)
        
        adjust_run_files(settings_run['dir_setup'],dir_run,settings_clm,settings_pfl,settings_sbatch)
    
        os.chdir(dir_run)
        start_run(settings_run['dir_build'])
        ## in tsmp_slm_run.bsh, a file ready.txt is written at the end of the run: wait for this file
        wait_for_run(dir_run,settings_sbatch) 
        os.chdir(settings_run['dir_setup'])
        remove_misc_files(dir_run)
        
        # ## Last step: move the run directory to storage (scratch), and keep a link
        # if not os.path.exists(settings_run['dir_store']):
        #     print('Creating dir: %s' % settings_run['dir_store'])
        #     os.makedirs(settings_run['dir_store'])
        # print('Moving files to storage: %s' % settings_run['dir_store'])
        # move_and_link(dir_run,settings_run['dir_store'])
        
            
    else:
        print('%s exists, continuing...' % dir_run) 

Continuing simulation in /p/project/cjibg36/kaandorp2/TSMP_patched/DA_tsmp_cordex_111x108
Preparing simulation in /p/project/cjibg36/kaandorp2/TSMP_patched/DA_tsmp_cordex_111x108/20190101-20191230/i000/R000/run_20190101-20190131
Copy binaries from /p/project/cjibg36/kaandorp2/TSMP_patched/bin/JUWELS_3.1.0MCT_clm-pfl



  Preparing the environment for use of requested stage ( 2020 ).


Currently Loaded Modules:
  1) Stages/2020                         (S)
  2) GCCcore/.9.3.0                      (H)
  3) zlib/.1.2.11                        (H)
  4) binutils/.2.34                      (H)
  5) Intel/2020.2.254-GCC-9.3.0
  6) numactl/2.0.13
  7) nvidia-driver/.default              (H,g)
  8) CUDA/11.0                           (g)
  9) UCX/1.9.0
 10) pscom/.5.4-default                  (H)
 11) XZ/.5.2.5                           (H)
 12) libxml2/.2.9.10                     (H)
 13) mpi-settings/UCX-plain
 14) ParaStationMPI/5.4.7-1              (g)
 15) imkl/2020.2.254
 16) Hypre/2.20.0
 17) Silo/4.10.2
 18) Tcl/8.6.10
 19) GMP/6.2.0
 20) nettle/.3.6                         (H)
 21) bzip2/.1.0.8                        (H)
 22) expat/.2.2.9                        (H)
 23) libpng/.1.6.37                      (H)
 24) freetype/.2.10.1                    (H)
 25) gperf/.3.1                          (H)
 2

Using process grid (6,6,1)
Using process grid (6,6,1)
/p/project/cjibg36/kaandorp2/TSMP_patched/parflow
Using process grid (6,6,1)
/p/project/cjibg36/kaandorp2/TSMP_patched/parflow
Using process grid (6,6,1)
/p/project/cjibg36/kaandorp2/TSMP_patched/parflow



  Preparing the environment for use of requested stage ( 2020 ).


Currently Loaded Modules:
  1) Stages/2020                         (S)
  2) GCCcore/.9.3.0                      (H)
  3) zlib/.1.2.11                        (H)
  4) binutils/.2.34                      (H)
  5) Intel/2020.2.254-GCC-9.3.0
  6) numactl/2.0.13
  7) nvidia-driver/.default              (H,g)
  8) CUDA/11.0                           (g)
  9) UCX/1.9.0
 10) pscom/.5.4-default                  (H)
 11) XZ/.5.2.5                           (H)
 12) libxml2/.2.9.10                     (H)
 13) mpi-settings/UCX-plain
 14) ParaStationMPI/5.4.7-1              (g)
 15) imkl/2020.2.254
 16) Hypre/2.20.0
 17) Silo/4.10.2
 18) Tcl/8.6.10
 19) GMP/6.2.0
 20) nettle/.3.6                         (H)
 21) bzip2/.1.0.8                        (H)
 22) expat/.2.2.9                        (H)
 23) libpng/.1.6.37                      (H)
 24) freetype/.2.10.1                    (H)
 25) gperf/.3.1                          (H)
 2

Submitted batch job 8149530
Still running...
Still running...
Still running...
Still running...
Still running...
Still running...
Still running...
Moving files to storage: /p/scratch/cjibg36/kaandorp2/TSMP_results/TSMP_patched/DA_tsmp_cordex_111x108


Error: Destination path '/p/scratch/cjibg36/kaandorp2/TSMP_results/TSMP_patched/DA_tsmp_cordex_111x108/run_20190101-20190131' already exists

In [9]:
print(dir_run)
print(settings_run['dir_store'])

/p/project/cjibg36/kaandorp2/TSMP_patched/DA_tsmp_cordex_111x108/20190101-20191230/i000/R000/run_20190101-20190131
/p/scratch/cjibg36/kaandorp2/TSMP_results/TSMP_patched/DA_tsmp_cordex_111x108


In [3]:

files = sorted(glob('/p/scratch/cjibg36/kaandorp2/TSMP_results/TSMP_patched/DA_tsmp_cordex_111x108/input_DA/*'))
data1 = np.load(files[1])
data2 = np.load(files[3])

In [4]:
data1

array([-0.26007095, -0.35771948, -0.30138268, ..., -2.56539904,
       -3.64451564, -0.95215511])

In [5]:
data2

array([-1.0559463 , -0.6404199 , -1.42397611, ..., -4.77487098,
       -0.66330751, -2.60930289])

In [11]:

[os.remove(file) for file in glob('/p/project/cjibg36/kaandorp2/TSMP_patched/tsmp_cordex_111x108_KsField/run_20190101-20191231/CLMSAT*.nc')]

[None]

In [ ]:
# This is working

date_start = datetime(2010,1,1,12,0,0)
date_end = datetime(2020,1,1,12,0,0)
freq_output = '3d'
freq_iter = 'YS'
freq_restart = 'MS'

date_iterations = pd.date_range(date_start,date_end,freq=freq_iter)
date_restarts = pd.date_range(date_start,date_end,freq=freq_restart)
date_results = pd.date_range(date_start,date_end,freq=freq_output)

date_iter_binned = bin_dates_by_restart_dates(date_results,date_iterations)
date_restarts_binned = bin_dates_by_restart_dates(date_restarts,date_iterations)
date_results_binned = [bin_dates_by_restart_dates(date_iter_binned[i1],date_restarts_binned[i1]) for i1 in range(len(date_iter_binned))] 